In [32]:
import pandas as pd
from Bio import SeqIO
from Bio.PDB import Vector
from Bio.PDB import PDBParser
from Bio.PDB import Polypeptide
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import warnings
from os import listdir
warnings.simplefilter('ignore')
import math
import pymol
import numpy as np
from Bio.PDB.Polypeptide import PPBuilder 

In [33]:
def readPDB(path):
    with open(path, "r") as f:
        return f.read()

In [42]:
target_dir = "../data/pdb_str/m32k25/"
files = [ "".join([target_dir, f]) for f in listdir(target_dir) ]
len(files)

30740

In [43]:
seq     = [ readPDB(f).rstrip('\n') for f in files ]
pdb_ids = [ f.split("/")[-1].split(".")[0] for f in files ]
d = {'pdb': pdb_ids, 'seq': seq}
df = pd.DataFrame(data=d)
df.head()

,pdb,seq
0,12asA00,UUUUUUUUUUUUUUVUUUUWWWOQDFBHDIMNBIFXSOSRJMDXOT...
1,132lA00,IKUUUUUUVSTPSSXMXYQCKVVUUUUVTVXKYQLUQEBGDKVPRD...
2,153lA00,TSWHHLVURCELDNCKVWWVUUPQLNBRXUUUUUUUUUUSUUUVVW...
3,155cA00,OMLVUVUVUVVUWTVWNGAQDKWHIMXFEHJTNIKLRXSTXRIHXR...
4,16pkA02,UUUUNHEHHXMAAAEEEHNJVUWSUUUVWWTUWRGNGADERYVVVU...


In [45]:
df.to_csv("../data/pdb_str/all_m32k25_cath.csv", index=False)

**Als Beispiel nehme ich die Atomsequenz des PDB 12as**

In [146]:
for record in SeqIO.parse("../data/pdb_str/atoms/12asA00", "fasta"):
    pdb = list(record.seq)

**Die Fragment Tabelle aus dem Paper wird für das Encoding verwendet:**

In [147]:
fragments = pd.DataFrame({
    "fragment" : ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y"],
    "phi1" : [122.4, 129.8, 117.1,118.4, 116.7,115.6,135.3,120.1,133.6,115.9,119.7,110.0,110.0,90.1,92.4,91.8,95.9,94.5,96.3,93.0,91.4,93.3,93.8,111.4,89.0 ],
    "phi2" : [119.4, 135.6, 111.0, 126.9,138.6,112.9,118.6,114.3,117.1,91.4,90.4,90.8,100.8,138.2,91.2,96.7,117.7,112.6,94.7, 92.8, 90.7, 89.1, 105.2, 94.6,95.1],
    "theta": [-164.2, -176.6, -142.2, -146.1,168.7,-117.9,-148.5,-90.7,-120.8,-134.6, -105.9,-158.8,177.0,19.6,-127.4,-104.8,136.0,115.0,112.0,83.1,49.8,68.3,32.3,21.8,-54.4]
})

def map2mk32k25(v):
    fragments["dist"] = fragments[["phi1","phi2","theta"]].apply(lambda x: np.linalg.norm(v - x), axis=1)
    return fragments.sort_values(by="dist").reset_index(drop=True).loc[0]["fragment"]

fragments

,fragment,phi1,phi2,theta
0,A,122.4,119.4,-164.2
1,B,129.8,135.6,-176.6
2,C,117.1,111.0,-142.2
3,D,118.4,126.9,-146.1
4,E,116.7,138.6,168.7
5,F,115.6,112.9,-117.9
6,G,135.3,118.6,-148.5
7,H,120.1,114.3,-90.7
8,I,133.6,117.1,-120.8
9,J,115.9,91.4,-134.6


**Konvertiere Atom Sequenz in das M32K25 Alphabet:**

In [152]:
source_dir = "../data/pdb_str/atoms/"
target_dir = "../data/pdb_str/M32K25/"
source_files = [ "".join([source_dir, f]) for f in listdir(source_dir)]
source_files

['../data/pdb_str/atoms/12asA00',
 '../data/pdb_str/atoms/132lA00',
 '../data/pdb_str/atoms/153lA00',
 '../data/pdb_str/atoms/155cA00',
 '../data/pdb_str/atoms/16pkA02',
 '../data/pdb_str/atoms/16vpA00',
 '../data/pdb_str/atoms/17gsA01',
 '../data/pdb_str/atoms/1914A00',
 '../data/pdb_str/atoms/19hcA01',
 '../data/pdb_str/atoms/19hcA02',
 '../data/pdb_str/atoms/1a00B00',
 '../data/pdb_str/atoms/1a02F00',
 '../data/pdb_str/atoms/1a04A01',
 '../data/pdb_str/atoms/1a04A02',
 '../data/pdb_str/atoms/1a0aA00',
 '../data/pdb_str/atoms/1a0cA00',
 '../data/pdb_str/atoms/1a0gA02',
 '../data/pdb_str/atoms/1a0hA01',
 '../data/pdb_str/atoms/1a0hA02',
 '../data/pdb_str/atoms/1a0iA01',
 '../data/pdb_str/atoms/1a0iA02',
 '../data/pdb_str/atoms/1a0iA03',
 '../data/pdb_str/atoms/1a0kA00',
 '../data/pdb_str/atoms/1a0pA01',
 '../data/pdb_str/atoms/1a0rB00',
 '../data/pdb_str/atoms/1a0rP01',
 '../data/pdb_str/atoms/1a0rP02',
 '../data/pdb_str/atoms/1a0sP00',
 '../data/pdb_str/atoms/1a10I00',
 '../data/pdb_

In [155]:
# https://biopython.org/DIST/docs/api/Bio.PDB.Polypeptide-pysrc.html
parser = PDBParser(PERMISSIVE=0)

# 1ojw
# 1ojx

for file in source_files:
    pdb_id = file.split("/")[-1]

    for model in PDBParser().get_structure(pdb_id, file):
        for chain in model:
            poly = Polypeptide.Polypeptide(chain)
            isCa = True
            try:
#             ca_list = [ p.get_vector()  for p in poly.get_ca_list()]
                tau_list = [ p * 180 / math.pi  for p in poly.get_tau_list()] # torsion angle
                theta_list = [ p * 180 / math.pi  for p in poly.get_theta_list()] # bond angle
                aa_sequence = [ p  for p in poly.get_sequence()]
            except: 
                icCa = False
    
    if isCa:
        df_angles = pd.DataFrame({"phi1"  : theta_list[0:-1],
                                  "phi2"  : theta_list[1:],
                                  "theta" : tau_list})

        df_angles["M32K25"] = df_angles.apply(lambda x: map2mk32k25(x.values) ,axis=1)
        m32k25 = "".join(df_angles["M32K25"].values)
        with open("%s%s" % (target_dir, pdb_id), "w") as o:
            SeqIO.write(SeqRecord(Seq(m32k25), id=pdb_id, description=""), o, "fasta")
    

**Erstelle phi-phi-theta Tabelle:  
  phi1 = das Vektor mit den Bindungswinkeln ohne das letzte Element  
  phi2 = das Vektor mit den Bindungswinkeln ohne das erste Element  
  theta = Torsionswinkel**

In [156]:
df_angles = pd.DataFrame({"phi1"  : theta_list[0:-1],
                          "phi2"  : theta_list[1:],
                          "theta" : tau_list})

df_angles.head()

,phi1,phi2,theta
0,111.465246,114.334799,-92.470635
1,114.334799,120.936283,-150.147530
2,120.936283,120.920902,-141.198853
3,120.920902,128.071391,-166.644083
4,128.071391,120.606886,-159.723223


**Kodierung**  
Die Atomsequenz des 12as wird nun in das Alphabet kodiert basierend auf der euklidischen Distanz. Und zwar wird zu jedem Symbol die Distanz berechnet und solches Symbol wird ausgewählt, welche die kleinste Distanz aufweist. Hier ein Beispiel für das este Atom, welches in das Symbol **U** kodiert wird.

In [126]:
import numpy as np
fragments["dist"] = fragments[["phi1","phi2","theta"]].apply(lambda x: np.linalg.norm(df_angles.loc[14] - x), axis=1)
fragments.sort_values(by="dist")

,fragment,phi1,phi2,theta,dist
21,V,93.3,89.1,68.3,8.135571
20,U,91.4,90.7,49.8,14.149979
19,T,93.0,92.8,83.1,22.305568
22,W,93.8,105.2,32.3,36.688628
23,X,111.4,94.6,21.8,47.346362
18,S,96.3,94.7,112.0,50.881939
17,R,94.5,112.6,115.0,59.521567
13,N,90.1,138.2,19.6,68.431961
16,Q,95.9,117.7,136.0,80.692102
12,M,110.0,100.8,177.0,117.376198


In [8]:
print("Anzahl der C-atome %s:" % df_angles.shape[0])

Anzahl der C-atome 324:


In [9]:
print("Anzahl der Zeichen Aminosäuresequenz %s:" % len(aa_sequence))


Anzahl der Zeichen Aminosäuresequenz 327:


In [15]:
sele = "12as"
# sele = "resi 1-327"
resis_resolved = 0
resis = 0
                                 
pymol.cmd.reinitialize()
pymol.cmd.load("../data/pdb_str/pdb/12as.ent")
resis_resolved = pymol.cmd.iterate_state(-1, sele + " and name ca", 'pass')
resis = pymol.cmd.iterate(sele + " and name ca", 'pass')

resis_resolved == resis

True

In [11]:
resis_resolved

656

In [12]:
resis

656

In [13]:
C:/Users/tomin/dev/data/pdb_str/atoms/12as.ent

SyntaxError: invalid syntax (<ipython-input-13-fd9f5159e4dd>, line 1)